In [10]:
import read as rd

import json
import pprint
import pandas as pd
catalog = pd.read_csv('./catalog.csv')

In [2]:
catalog_dict = rd.read_catalog(catalog)

Retrieving Data for Model H
Retrieving Data for Model B
Retrieving Data for Model A
Retrieving Data for Model R
Retrieving Data for W18
Retrieving Data for W19
Retrieving Data for W20
Retrieving Data for Black Seat
Retrieving Data for White Seat


In [18]:
pprint.pprint(catalog_dict)

{'Model': {'Model A': {'addr': 'DFW International Airport (DFW), 2400 Aviation '
                               'Dr, DFW Airport, TX 75261, USA',
                       'manufacturer': 'Hedera Motors Dallas',
                       'materials': {'metal': 1300, 'plastic': 700},
                       'name': 'Model A',
                       'options': {'seats': None, 'wheels': None},
                       'price': 50000,
                       'weight': 1800},
           'Model B': {'addr': '6CPQ+9HC, Xiaoshan District, Hangzhou, '
                               'Zhejiang, China',
                       'manufacturer': 'Hedera Motors Hangzhou',
                       'materials': {'metal': 1100, 'plastic': 500},
                       'name': 'Model B',
                       'options': {'seats': None, 'wheels': None},
                       'price': 70000,
                       'weight': 1800},
           'Model H': {'addr': 'DFW International Airport (DFW), 2400 Aviation '
        

In [19]:
with open('./options.json', 'w') as f:
    json.dump(catalog_dict, f)

### Loading the json

In [20]:
with open('./options.json') as f:
    data = json.load(f)
data

{'Model': {'Model H': {'name': 'Model H',
   'price': 40000,
   'manufacturer': 'Hedera Motors Dallas',
   'addr': 'DFW International Airport (DFW), 2400 Aviation Dr, DFW Airport, TX 75261, USA',
   'weight': 1800,
   'materials': {'metal': 1000, 'plastic': 400},
   'options': {'seats': None, 'wheels': None}},
  'Model B': {'name': 'Model B',
   'price': 70000,
   'manufacturer': 'Hedera Motors Hangzhou',
   'addr': '6CPQ+9HC, Xiaoshan District, Hangzhou, Zhejiang, China',
   'weight': 1800,
   'materials': {'metal': 1100, 'plastic': 500},
   'options': {'seats': None, 'wheels': None}},
  'Model A': {'name': 'Model A',
   'price': 50000,
   'manufacturer': 'Hedera Motors Dallas',
   'addr': 'DFW International Airport (DFW), 2400 Aviation Dr, DFW Airport, TX 75261, USA',
   'weight': 1800,
   'materials': {'metal': 1300, 'plastic': 700},
   'options': {'seats': None, 'wheels': None}},
  'Model R': {'name': 'Model R',
   'price': 80000,
   'manufacturer': 'Hedera Motors Hangzhou',
   'ad

In [463]:
# requirements.txt
# json
# pydeck
# haversine
# googlemaps

In [17]:
import sys
import json
import pydeck as pdk
import haversine as hs

local = sys.modules[__name__]

In [19]:
from treehouse.options.seats import seats
from treehouse.options.wheels import wheels
from treehouse.options.models import models

In [4]:
import retrieve as rcv 

In [5]:
class Configurator:

    def __init__(self):
        self.car = None
        self.kgm = 0
        self.mat = {}
        self.nodes = []
        self.edges = []

    def select_model(self, model):
        self.car = json.loads(json.dumps(
            getattr(local, 'models')[model]
        ))

    def select_option(self, option, choice):
        self.car['options'][option] = json.loads(json.dumps(
            getattr(local, option)[choice]
        ))

    def get_coords(self, addr):

        response = gmaps.geocode(addr)

        return response[0]['geometry']['location']['lat'], response[0]['geometry']['location']['lng']  

    def set_coords(self, parent=None):
        
        if parent is None:
            parent = self.car
        
        parent['lat'], parent['lng'] = self.get_coords(parent['addr'])
        
        if 'options' in parent:
            for option, child in parent['options'].items():
                self.set_coords(child)
                
    def sum_mat(self, parent=None):
        
        if parent is None:
            parent = self.car
            self.mat = {}
            
        if 'materials' in parent:
            for material, amt in parent['materials'].items():
                if material not in self.mat:
                    self.mat[material] = 0
                self.mat[material] += amt
            
        if 'options' in parent:
            for option, child in parent['options'].items():
                self.sum_mat(child)
                
    def get_cumul_weight(self, parent=None):
        
        if parent is None:
            parent = self.car
            self.edges = []

        if 'cumul_weight' not in parent:
            parent['cumul_weight'] = parent['weight']

        if 'options' in parent:
            for option, child in parent['options'].items():
                parent['cumul_weight'] += child['weight']
                self.get_cumul_weight(child)

    def get_nodes(self, parent=None):
        
        if parent is None:
            parent = self.car
            self.nodes = []
        
        self.nodes.append({k: parent[k] for k in set(list(parent.keys())) - set(['options'])})

        if 'options' in parent:
            for option, child in parent['options'].items():
                self.get_nodes(child)
                
    def calc_edges(self, parent=None):
        
        if parent is None:
            parent = self.car
            self.edges = []

        if 'options' in parent:
            for option, child in parent['options'].items():

                parent_coords = [parent['lng'], parent['lat']]
                child_coords = [child['lng'], child['lat']]

                self.edges.append({
                    'start': parent_coords,
                    'end': child_coords,
                    'name': f"{child['name']} -> {parent['name']}",
                    'cumul_weight': child['cumul_weight'],
                    'distance': hs.haversine(
                        parent_coords[::-1],
                        child_coords[::-1])
                })

            self.calc_edges(child)
            
    def get_kgm(self):
        
        for edge in self.edges:
            self.kgm += edge['cumul_weight'] * edge['distance']
            
    def plot(self):
        
        layer = pdk.Layer(
            "ScatterplotLayer",
            data=car.nodes,
            get_position="[lng, lat]",
            get_color=[255, 0, 0],  # Set color to red
            radius_scale=1,
            radius_min_pixels=5,
            radius_max_pixels=10,
            get_radius=10000,  # Set radius to 100 pixels
            pickable=True,
        )

        line_layer = pdk.Layer(
            "LineLayer",
            car.edges,
            get_source_position="start",
            get_target_position="end",
            get_color=[255, 255, 255],
            get_width=3,
            highlight_color=[255, 255, 255],
            picking_radius=10,
            auto_highlight=True,
            pickable=True,
        )

        view = pdk.ViewState(latitude=30, longitude=0, min_zoom=0.7, zoom=1.5, max_zoom=4)

        deck = pdk.Deck(layers=[line_layer, layer], initial_view_state=view)
        
        return deck

In [20]:
dir(query_result)

['__class__',
 '__cls_storage',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__javaclass__',
 '__javaconstructor__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_class',
 'asBytes',
 'bloom',
 'clone',
 'contractFunctionParametersBytes',
 'contractId',
 'createdContractIds',
 'equals',
 'errorMessage',
 'errorPrefix',
 'evmAddress',
 'finalize',
 'gas',
 'gasUsed',
 'getAddress',
 'getBool',
 'getByteBuffer',
 'getByteString',
 'getBytes',
 'getBytes32',
 'getClass',
 'getDynamicBytes',
 'getInt256',
 'getInt32',
 'getInt64',
 'getInt8',
 'getIntValueAt',
 'getString',
 'getStringArray',
 'getUint256',
 'getUint32',
 'getUint64',
 'getUint8',
 'hashCode',
 'hbarAmount',
 'logs',
 'not

In [6]:
car = Configurator()

In [7]:
car.select_model('Model A') # to be a dropdown

In [8]:
car.select_option('seats', 'White Seat') # to be a dropdown

In [9]:
car.select_option('wheels', 'W19') # to be a dropdown

In [10]:
car.set_coords() # set coords of all addresses

In [11]:
car.sum_mat() # sum materials

In [12]:
car.get_cumul_weight()

In [13]:
car.get_nodes() # hi

In [14]:
car.calc_edges() # artificial intelligence blockchain big data quantum computing

In [15]:
car.plot()

{
  "initialViewState": {
    "latitude": 30,
    "longitude": 0,
    "maxZoom": 4,
    "minZoom": 0.7,
    "zoom": 1.5
  },
  "layers": [
    {
      "@@type": "LineLayer",
      "autoHighlight": true,
      "data": [
        {
          "cumul_weight": 100,
          "distance": 8396.641135536738,
          "end": [
            13.510658,
            52.364628
          ],
          "name": "W19 -> Model A",
          "start": [
            -97.0403352,
            32.8998091
          ]
        },
        {
          "cumul_weight": 100,
          "distance": 1510.6163173753025,
          "end": [
            -99.0805443,
            19.4375028
          ],
          "name": "White Seat -> Model A",
          "start": [
            -97.0403352,
            32.8998091
          ]
        }
      ],
      "getColor": [
        255,
        255,
        255
      ],
      "getSourcePosition": "@@=start",
      "getTargetPosition": "@@=end",
      "getWidth": 3,
      "highlightColor": [
        255,
        255,
        255
      ],
      "id": "513a18dd-3cd0-4879-a9cb-a22eeaf3069e",
      "pickable": true,
      "pickingRadius": 10
    },
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "addr": "2400 Aviation Dr, DFW Airport, TX 75261, USA",
          "cumul_weight": 2300,
          "lat": 32.8998091,
          "lng": -97.0403352,
          "manufacturer": "Hedera Motors Dallas",
          "materials": {
            "metal": 1300,
            "plastic": 700
          },
          "name": "Model A",
          "price": 50000,
          "weight": 2100
        },
        {
          "addr": "Melli-Beese-Ring 1, 12529 Sch\u00f6nefeld, Germany",
          "cumul_weight": 100,
          "lat": 52.364628,
          "lng": 13.510658,
          "manufacturer": "Berlin Wheel Company Ltd",
          "materials": {
            "metal": 38,
            "rubber": 11
          },
          "name": "W19",
          "price": 1000,
          "weight": 100
        },
        {
          "addr": "Av. Capitan Carlos Leon S/N, Penon de los Banos, Venustiano Carranza, 15620 Ciudad de Mexico, CDMX, Mexico",
          "cumul_weight": 100,
          "lat": 19.4375028,
          "lng": -99.0805443,
          "manufacturer": "Mexico City Seat Company",
          "materials": {
            "leatherette": 15,
            "metal": 20
          },
          "name": "White Seat",
          "price": 1000,
          "weight": 100
        }
      ],
      "getColor": [
        255,
        0,
        0
      ],
      "getPosition": "@@=[lng, lat]",
      "getRadius": 10000,
      "id": "846f45dd-0141-49fb-8e4a-e87e118a6826",
      "pickable": true,
      "radiusMaxPixels": 10,
      "radiusMinPixels": 5,
      "radiusScale": 1
    }
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}